In [1]:
import tensorflow as tf
import numpy as np
import argparse
import facenet
import os
import sys
import math
import pickle, pdb
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Conv2D, Input, Reshape, Dense, Flatten, Dropout
from keras.utils import to_categorical

data_dir = "../datasets/lfw/lfw_mtcnnpy_gt6"
model_dir = "../models/20180408-102900/20180408-102900.pb"
batch_size = 90
image_size = 160

def split_dataset(dataset, min_nrof_images_per_class, nrof_train_images_per_class, nrof_test_images_per_class):
    train_set = []
    test_set = []
    for cls in dataset:
        paths = cls.image_paths
        # Remove classes with less than min_nrof_images_per_class
        if len(paths)>=min_nrof_images_per_class:
            np.random.shuffle(paths)
            train_set.append(facenet.ImageClass(cls.name, paths[:nrof_train_images_per_class]))
            test_set.append(facenet.ImageClass(cls.name, paths[nrof_train_images_per_class:nrof_train_images_per_class+nrof_test_images_per_class]))
    return train_set, test_set

def get_embeddings(dataset):
    paths, labels = facenet.get_image_paths_and_labels(dataset)
    print('Number of classes: %d' % len(dataset))
    print('Number of images: %d' % len(paths))

    # Load the model
    print('Loading feature extraction model')
    facenet.load_model(model_dir)

    # Get input and output tensors
    images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
    embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
    phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
    embedding_size = embeddings.get_shape()[1]

    # Run forward pass to calculate embeddings
    print('Calculating features for images')
    nrof_images = len(paths)
    nrof_batches_per_epoch = int(math.ceil(1.0*nrof_images / batch_size))
    emb_array = np.zeros((nrof_images, embedding_size))

    for i in range(nrof_batches_per_epoch):
        start_index = i*batch_size
        end_index = min((i+1)*batch_size, nrof_images)
        paths_batch = paths[start_index:end_index]
        images = facenet.load_data(paths_batch, False, False, image_size)
        feed_dict = { images_placeholder:images, phase_train_placeholder:False }
        emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)

    return emb_array, labels

Using TensorFlow backend.


In [1007]:
## Calculate face embeddings from dataset
with tf.Graph().as_default():
    with tf.Session() as sess:
        dataset = facenet.get_dataset(data_dir)
        train_set, test_set = split_dataset(dataset, 8, 7, 1)

        train_embs, train_labels = get_embeddings(train_set)
        test_embs, test_labels = get_embeddings(test_set)

Number of classes: 217
Number of images: 1519
Loading feature extraction model
Model filename: ../models/20180408-102900/20180408-102900.pb
Calculating features for images
Number of classes: 217
Number of images: 217
Loading feature extraction model
Model filename: ../models/20180408-102900/20180408-102900.pb
Calculating features for images


In [279]:
# get a specific number of training images per person
def split_train_dataset(nb_images_per_person):
    i, train_embs, train_labels = 0, [], []
    for l in range(len(orig_train_labels)):
        if (l+1 < len(orig_train_labels) and orig_train_labels[l+1] != orig_train_labels[l]) or l+1 == len(orig_train_labels):
            train_embs.extend(orig_train_embs[i:i+nb_images_per_person])
            train_labels.extend(orig_train_labels[i:i+nb_images_per_person])
            i = l+1
    return np.array(train_embs), train_labels

# read train/test embs/labels from pickle file
with open("lfw_embs/2_per_face.pkl", 'rb') as handle:
    orig_train_embs, orig_train_labels, test_embs, test_labels, label_embs = pickle.load(handle)

assert len(set(orig_train_labels)) == len(set(test_labels))

train_embs, train_labels = split_train_dataset(nb_images_per_person=2)

print("Nb of classes:", len(set(test_labels)))
print("Nb of train faces per class:", len(train_labels)/len(set(test_labels)))
print("Nb of test faces per class:", len(test_labels) / len(set(test_labels)))

Nb of classes: 901
Nb of train faces per class: 2.0
Nb of test faces per class: 1.0


In [276]:
get_euclidean_acc(train_embs, train_labels, test_embs, test_labels)

Train accuracy: 0.9994450610432852
Test accuracy: 0.9400665926748057


#### Get accuracy of the classifier

In [277]:
model = SVC(kernel='linear', probability=True)#, gamma=0.001)
model = MLPClassifier(hidden_layer_sizes=(1000))
model.fit(train_embs, train_labels)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=1000, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [278]:
predictions = model.predict_proba(train_embs)
print("Train accuracy:", model.score(train_embs, train_labels))

predictions = model.predict_proba(test_embs)
print("Test accuracy:", np.mean(np.equal(np.argmax(predictions, axis=1), test_labels)))

Train accuracy: 0.9994450610432852
Test accuracy: 0.904550499445061


In [263]:
# # Create a list of class names
# class_names = [ cls.name.replace('_', ' ') for cls in dataset]
# # Saving classifier model
# with open("lwf.pkl", 'wb') as outfile:
#     pickle.dump((model, class_names), outfile)
# print('Saved classifier model to file "%s"' % 'lwf.pkl')

#### Get accuracy of the average euclidean distance method

In [91]:
label_embs = []

def get_euclidean_acc(train_embs, train_labels, test_embs, test_labels):
    i, label_embs = 0, []
    while i < len(train_labels):
        label_emb = np.copy(train_embs[i])
        init_pos = i
        while i+1 < len(train_labels) and train_labels[i] == train_labels[i+1]:
            label_emb += train_embs[i+1]
            i+=1
        label_emb /= (i - init_pos + 1)
        # normalize the vector
        label_embs.append(label_emb / np.linalg.norm(label_emb))
        i+=1
    label_embs = np.array(label_embs)

    for embs, labels, name in zip([train_embs, test_embs], [train_labels, test_labels], ['Train', 'Test']):
        predictions = np.argmax(embs.dot(label_embs.T), axis=1)
        accuracy = np.mean(np.equal(predictions, labels))
        print(name, "accuracy:", str(accuracy))

In [280]:
np.where((test_embs.dot(label_embs.T).argmax(axis=1) == test_labels) == False)

(array([ 29,  85,  98, 105, 113, 118, 126, 129, 134, 135, 143, 152, 160,
        163, 168, 175, 194, 215, 222, 226, 237, 238, 249, 252, 277, 284,
        297, 327, 342, 343, 369, 381, 394, 409, 437, 453, 458, 463, 472,
        483, 485, 492, 503, 506, 527, 553, 559, 563, 599, 607, 608, 613,
        634, 673, 708, 748, 764, 765, 777, 791, 795, 797, 804, 807, 836,
        838, 849, 860, 899]),)

#### Experimenting with deep learning

In [281]:
# train_embs = np.loadtxt('lfw_embs/2_per_face_train.out')
# test_embs = np.loadtxt('lfw_embs/2_per_face_test.out')
# label_embs = np.loadtxt('lfw_embs/2_per_face_train_avg.out')
# train_embs = scaler.fit_transform(train_embs)
# test_embs = scaler.fit_transform(test_embs)
# label_embs = scaler.fit_transform(label_embs)

X_train, y_train = train_embs, to_categorical(train_labels)
X_test, y_test = test_embs, to_categorical(test_labels)

In [282]:
inputs = Input(shape=(512,))
x = Dense(1000, activation='relu')(inputs)
x = Dropout(0.9)(x)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.9)(x)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.9)(x)
x = Dense(1000, activation='relu')(inputs)
predictions = Dense(y_train.shape[1], activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [290]:
model.fit(X_train, y_train, epochs=15, batch_size=64)

score, acc = model.evaluate(X_test, y_test)#,batch_size=batch_size)
print('Test score:', score, '\nTest accuracy:', acc)

Epoch 1/15
1802/1802 [==============================] - 1s 388us/step - loss: 7.5124e-07 - acc: 1.0000
Epoch 2/15
1802/1802 [==============================] - 1s 336us/step - loss: 7.3566e-07 - acc: 1.0000
Epoch 3/15
1802/1802 [==============================] - 1s 336us/step - loss: 7.2511e-07 - acc: 1.0000
Epoch 4/15
1802/1802 [==============================] - 1s 338us/step - loss: 7.0785e-07 - acc: 1.0000
Epoch 5/15
1802/1802 [==============================] - 1s 329us/step - loss: 6.9667e-07 - acc: 1.0000
Epoch 6/15
1802/1802 [==============================] - 1s 333us/step - loss: 6.8850e-07 - acc: 1.0000
Epoch 7/15
1802/1802 [==============================] - 1s 340us/step - loss: 6.7656e-07 - acc: 1.0000
Epoch 8/15
1802/1802 [==============================] - 1s 335us/step - loss: 6.5946e-07 - acc: 1.0000
Epoch 9/15
1802/1802 [==============================] - 1s 337us/step - loss: 6.5013e-07 - acc: 1.0000
Epoch 10/15
1802/1802 [==============================] - 1s 338us/step - 

In [255]:
%%timeit
model.predict(X_train[1].reshape(1,-1)).argmax()

327 µs ± 54.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [1012]:
# # write pickle file
# with open("lfw_embs/7_per_face.pkl", 'wb') as outfile:
#     pickle.dump((train_embs, train_labels, test_embs, test_labels, label_embs), outfile)
# print('Saved classifier model to file "%s"' % outfile)

Saved classifier model to file "<_io.BufferedWriter name='lfw_embs/7_per_face.pkl'>"
